<a href="https://colab.research.google.com/github/butchland/build-your-own-image-classifier/blob/master/colab-build-image-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build your Image Classifier

## Instructions

1. In the **Specify Project Name** section below, fill out the project name first _(this is the name of the project you used in the previous notebook. If you didn't change the name of the default project in the previous notebook, you shouldn't have to change the default project name here either so just leave the project name as is)_.

    This notebook assumes that you have already cleaned your image dataset (i.e. the _`cleaned_pets.tgz`_ or its equivalent has already been saved to your Google Drive under the _`/My Drive/build-your-own-image-classifier/data/pets`_ directory or its equivalent.

    If the cleaned dataset does not exist, this will trigger an error. Please make sure to run the previous notebook ([Clean Images from your Image Dataset](https://colab.research.google.com/github/butchland/build-your-own-image-classifier/blob/master/colab-clean-image-dataset.ipynb)) before running this one. 

1. Click on the `Connect` button on the top right area of the page. This will change into a checkmark with the RAM and Disk health bars once the connection is complete.
1. Press `Cmd/Ctrl+F9` or Click on the menu `Runtime/Run all`
1. Click on the link to `accounts.google.com` that appears and login in to your Google Account if neccessary or select the Google Account to use for your Google Drive. (This will open a new tab)
1. Authorize `Google Drive File Stream` to access your Google Drive (We will use this to save your exported image classifier to a folder on your Google Drive).

1. Copy the generated authentication token and paste it on the input box that appears.

1. Once the text 'DONE! DONE! DONE!' is printed at the end of the notebook, You can click on the menu `Runtime/Factory reset runtime` and click `Yes` on the dialog box to end your session.

Your exported image classifier(`export.pkl`) will be saved in your Google Drive under `/My Drive/build-your-own-image-classifier/models/<project-name>`  _(if you didn't change the defaults, it should be under `/My Drive/build-your-own-image-classifier/models/pets/export.pkl`)_


## What is going on?

This section explains the code behind this notebook

_(Click on SHOW CODE to display the code)_

### Connect to your Google

We'll need to connect to your Google Drive in order to retrieve our cleaned images dataset as well as save the exported image classifier file afterwards.

In [ ]:
#@title {display-mode: "form"}
from google.colab import drive
drive.mount('/content/drive')

### Specify Project Name and Dataset Type

Fill out the `project name` -- the project name should be the same one used as the project name used in the previous notebook. 



In [ ]:
#@title Enter your project name {display-mode: "form"}
project = "pets" #@param {type: "string"}

### Install Python Packages

Install all the python packages to collect the images

In [ ]:
#@title {display-mode: "form"} 
!pip install -Uqq fastai --upgrade

### Copy your Image Dataset from Google Drive

In [ ]:
#@title {display-mode: "form"} 
file_name = f'cleaned_{project}.tgz'
folder_path = f'build-your-own-image-classifier/data/{project}'  
!cp /content/drive/My\ Drive/{folder_path}/{file_name} /content/.
!tar -xzf {file_name}

In [ ]:
#@title {display-mode: "form"} 
from fastai.vision.all import *
from fastai.vision.widgets import *
path = Path(f'/content/{project}')
Path.BASE_PATH = path
datablock = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=RandomSplitter(seed=42),
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224, min_scale=0.8)
)

In [ ]:
#@title {display-mode: "form"}
dls = datablock.dataloaders(path)
# dls.show_batch()

### Build your image classifier   

We now create the final version of our image classifier that will be used for our application.

In [ ]:
#@title {display-mode: "form"}
learn = cnn_learner(dls, resnet34, metrics=accuracy)

This just trains the image classifier on our images. 

The **accuracy** (times 100) is the percentage at which our final version of the image classifier correctly predicted the categories on our validation set _(validation metrics are used as a guide on how good our image classifier will be when deployed as an app)_

Notice that the training runs longer than the previous run -- that's because we are trying to get the best performance for our image classifier once its deployed.


In [ ]:
#@title {display-mode: "form"}
learn.fine_tune(10, freeze_epochs=4)

In [ ]:
#@title {display-mode: "form"}
learn.export()
model_path = f'build-your-own-image-classifier/models/{project}'
!mkdir -p /content/drive/My\ Drive/{model_path}
!cp /content/export.pkl /content/drive/My\ Drive/{model_path}
print("DONE! DONE! DONE!")
print(f'Your exported image classifier(export.pkl) is saved in your Google Drive under the folder My Drive/{model_path}/export.pkl')
print("Make sure to end your session (Click on menu Runtime/Factory reset runtime and click 'Yes' on the dialog box to end your session)")
print("before closing this notebook.")